# Geodésicas en métrica Kerr + Corrección EDR (toy)
Integración numérica de geodésicas en un fondo Kerr (coordenadas equatoriales) comparando trayectorias GR vs EDR.
Sustituye las piezas `metric_kerr_toy` y `EDR_potential_correction` por las funciones derivadas si las tienes.

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 120

## 1) Métrica Kerr simplificada (ecuatorial, toy)
Usamos una métrica simplificada en el plano ecuatorial ($\theta=\pi/2$) para integrar r(λ), φ(λ) con energía E y momento angular L conservados.

In [ ]:
def metric_kerr_toy(r, a, M=1.0):
    # devuelve g_tt, g_tphi, g_phiphi, g_rr para theta=pi/2
    Sigma = r**2
    Delta = r**2 - 2*M*r + a**2
    g_tt = -(1 - 2*M*r / Sigma)
    g_tphi = -2*M*a*r / Sigma
    g_phiphi = (r**2 + a**2 + 2*M*a**2*r / Sigma)
    g_rr = Sigma / Delta
    return g_tt, g_tphi, g_phiphi, g_rr

## 2) Corrección EDR a potencial efectivo radial (toy)
Usamos δV(r) que modifica la ecuación efectiva radial del movimiento geodésico.

In [ ]:
def EDR_potential_correction(r, params):
    # función suave y decreciente: pequeña corrección en la región r~r_H..10 r_H
    rH = params.get('rH', 1.0)
    amp = params.get('amp', 0.02)
    return amp * np.exp(-(r - rH)/8.0)


## 3) Ecuaciones efectivas (variables de primer orden)
Usamos coordenadas (t, r, phi) y escribimos ecuaciones para r'(λ), φ'(λ) usando conservación de E y L.
Implementamos integración numérica para probar trayectorias de prueba.

In [ ]:
def radial_eqs(y, lam, params):
    # y = [r, pr, phi]
    r, pr, phi = y
    M = params.get('M',1.0)
    a = params.get('a',0.5)
    E = params.get('E',0.95)
    L = params.get('L',3.5)
    # potencial efectivo simple V_eff = f(r;E,L) + deltaV
    V_eff = (1 - 2*M/r) * (1 + L**2 / r**2)
    V_eff_mod = V_eff + EDR_potential_correction(r, params)
    # ecuaciones: dr/dλ = pr, dpr/dλ = -dV_eff_mod/dr, dφ/dλ = L/r^2
    dVdr = (2*M/r**2)*(1 + L**2/r**2) + (1 - 2*M/r)*(-2*L**2 / r**3)
    dVdr_mod = dVdr + (-EDR_potential_correction(r, params)/8.0)
    dr_dlam = pr
    dpr_dlam = -dVdr_mod
    dphi_dlam = L / r**2
    return [dr_dlam, dpr_dlam, dphi_dlam]


## 4) Integración de ejemplo: GR vs EDR

In [ ]:
params_gr = {'M':1.0, 'a':0.5, 'E':0.95, 'L':3.5, 'rH':1.0, 'amp':0.0}
params_edr = {'M':1.0, 'a':0.5, 'E':0.95, 'L':3.5, 'rH':1.0, 'amp':0.03}

y0 = [10.0, 0.0, 0.0]
lam = np.linspace(0, 200, 5000)
sol_gr = odeint(radial_eqs, y0, lam, args=(params_gr,))
sol_edr = odeint(radial_eqs, y0, lam, args=(params_edr,))

r_gr = sol_gr[:,0]
phi_gr = sol_gr[:,2]
x_gr = r_gr * np.cos(phi_gr)
y_gr = r_gr * np.sin(phi_gr)

r_edr = sol_edr[:,0]
phi_edr = sol_edr[:,2]
x_edr = r_edr * np.cos(phi_edr)
y_edr = r_edr * np.sin(phi_edr)

plt.figure(figsize=(6,6))
plt.plot(x_gr, y_gr, label='GR')
plt.plot(x_edr, y_edr, label='EDR', linestyle='--')
plt.scatter([0],[0], color='k')
plt.xlim(-12,12); plt.ylim(-12,12)
plt.legend(); plt.title('Trayectorias GR vs EDR (toy)')
plt.gca().set_aspect('equal', 'box')
plt.show()

## 5) Observaciones
- Este notebook usa un modelo toy para la corrección EDR; sustituye `EDR_potential_correction` por la proyección angular/δV derivada para obtener resultados físicamente relevantes.
- Las trayectorias muestran pequeñas desviaciones en regiones cercanas al BH, que es donde δV es mayor.